In [ ]:
import numpy as np
import pandas as pd
import plotly.io as pio
import vectorbtpro as vbt
from vctr.data.lstm_preprocessor import preprocess_data_2d

# pio.renderers.default = 'notebook'
from vctr.data.data_loader import get_data_with_features_and_labels
from vctr.models.rfc_binary import RFClassifier


In [ ]:
symbol = 'MATIC'
timeframe = '30m'
freq = timeframe.replace('m', 't')

X, y = get_data_with_features_and_labels(symbol, timeframe=timeframe)
data = X.join(y)
X_train, y_train, X_test, y_test = preprocess_data_2d(X, y)
X_train.shape, y_train.shape, X_test.shape, y_test.shape


In [ ]:
rfc = RFClassifier()
rfc.train_and_evaluate(X_train, y_train, X_test, y_test)


In [ ]:
rfc.get_feature_importance(X)

In [ ]:
y_pred = pd.Series(rfc.model.predict(X_test), index=y_test.index)
data['pred'] = y_pred
data.dropna(inplace=True)

data


In [36]:
import vectorbtpro as vbt

def get_return(df):
    pf = vbt.Portfolio.from_signals(
        df['close'],
        df['label'] == 1,
        df['label'] == 0,
        freq='1H',
        init_cash=10000,
        fees=0.001,
        fixed_fees=0.001,
        slippage=0.001,
        log=True,
    )
    pf2 = vbt.Portfolio.from_signals(
        df['close'],
        df['pred'] == 0,
        df['pred'] == 1,
        freq='1H',
        init_cash=10000,
        fees=0.001,
        fixed_fees=0.001,
        slippage=0.001,
        log=True,
    )
    print(f'[true] ROI: {pf.stats("total_return").values[0]:.0f}%')
    print(f'[pred] ROI: {pf2.stats("total_return").values[0]:.0f}%')


In [37]:
get_return(data)

[true] ROI: 447897%
[pred] ROI: -97%


In [ ]:
len(data)